In [ ]:
# import the project directory here to find the emtl package
import os, sys
project_dir = os.path.abspath('..')

# if the kernel wasn't restarted, the folder might still be there
if project_dir not in sys.path: 
    sys.path.append(project_dir)

In [ ]:
import torch
from torchvision import models as M
from torchvision import datasets as D
from torchvision import transforms as T

# EMTL Library Imports
from emtl.tasks import SimpleTask
from emtl.algorithms import SequentialTraining
from emtl.trainer import Trainer

device = 'cuda'

Type of target to use, attr, identity, bbox, or landmarks. Can also be a list to output a tuple with all specified target types. The targets represent:
- attr (Tensor shape=(40,) dtype=int): binary (0, 1) labels for attributes
- identity (int): label for each person (data points with the same identity are the same person)
- bbox (Tensor shape=(4,) dtype=int): bounding box (x, y, width, height)
- landmarks (Tensor shape=(10,) dtype=int): landmark points (lefteye_x, lefteye_y, righteye_x, righteye_y, nose_x, nose_y, leftmouth_x, leftmouth_y, rightmouth_x, rightmouth_y)

In [ ]:
testset = D.CelebA(root='../data', split='valid', target_type='attr', transform=T.ToTensor(), target_transform=lambda x:x.float(), download=False)

In [ ]:
len(testset)

In [ ]:
class Head(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = torch.nn.Linear(2048, 40)

    def forward(self, x: torch.Tensor, **kwargs):
        return self.model(x)

In [ ]:
# make a ResNet50 backbone removing the last two layers (fc and avgpool)
backbone = M.resnet50(replace_stride_with_dilation=[False, True, True]).to(device)
backbone = torch.nn.Sequential(*list(backbone.children())[:-2], torch.nn.AdaptiveAvgPool2d(1), torch.nn.Flatten())
head = Head()

In [ ]:
from torch.utils.data import Subset

attributes_task = SimpleTask(
    name = 'CelebA',
    head = head,
    trainset = testset, 
    testset = Subset(testset, range(1000)),
    criterion = torch.nn.BCEWithLogitsLoss(reduction='sum'),
    optimizer_fn = torch.optim.Adam,
    scheduler_fn = torch.optim.lr_scheduler.ReduceLROnPlateau,
    config = '../configs/tasks/CelebA.ini'
)

In [ ]:
trainer = Trainer(
    backbone = backbone,
    tasks = [attributes_task],
    algorithm = SequentialTraining(epochs=5),
    config='../config.ini'
)

# train the model
trainer.launch()